In [104]:
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from ipyfilechooser import FileChooser
import pyvisa
import numpy as np

In [105]:
def device_list():
    rm = pyvisa.ResourceManager()
    for device_address in rm.list_resources():
        print(device_address)

def identity():
    global device_address
    rm = pyvisa.ResourceManager()
    with rm.open_resource(device_address) as instr:
        print(instr.query('*IDN?'))

def write():
    global device_address
    global command
    rm = pyvisa.ResourceManager()
    with rm.open_resource(device_address) as instr:
        print('>'+command)
        print(instr.write(command))

def query():
    global device_address
    global command
    global result
    rm = pyvisa.ResourceManager()
    with rm.open_resource(device_address) as instr:
        print('>'+command)
        result = instr.query(command)
        print(result)

def result_plot():
    global result
    plt.plot(list(map(float,result.split(','))))
    plt.show()

In [108]:
from optparse import Option


def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_device_list = widgets.Button(description='デバイスリスト')
    button_identity = widgets.Button(description='IDN')
    button_write = widgets.Button(description='write')
    button_query = widgets.Button(description='query')
    button_result_plot = widgets.Button(description='簡易プロット')
    

    filechooser = FileChooser('./')
    text_address = widgets.Combobox(value='',placeholder='visa',description='アドレス',disabled=False, layout= widgets.Layout(width='100%'))
    text_command = widgets.Combobox(value='',placeholder='scip',description='コマンド',disabled=False, layout= widgets.Layout(width='100%') )

    def add_address():
        rm = pyvisa.ResourceManager()
        text_address.options = tuple(rm.list_resources())

    def add_history():
        text_command.options += (text_command.value,)

    output = widgets.Output(layour={'border': '1px solid black'})
    def wrapped_func_factory(func):
        if type(func) is not list: func = [func]
        def new_func(ui_element):
            with output:
                for f in func:
                    print(f"exec func {f.__name__}")
                    f()
                    print(f"complete {f.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_device_list.on_click(wrapped_func_factory([add_address,device_list]))
    button_identity.on_click(wrapped_func_factory(identity))
    button_write.on_click(wrapped_func_factory([add_history, write]))
    button_query.on_click(wrapped_func_factory([add_history, query]))
    button_result_plot.on_click(wrapped_func_factory(result_plot))

    
    def load_input_field():
        global device_address
        global command
        device_address = text_address.value
        command = text_command.value
    text_command.observe(lambda element: load_input_field())
    text_address.observe(lambda element: load_input_field())
    
    display(
        widgets.HBox([button_clear_output,button_device_list,button_identity,button_write, button_query, button_result_plot]),
        widgets.HBox([filechooser,  ]),
        widgets.HBox([text_address]),
        widgets.HBox([text_command]),
        output)

In [109]:
simple_ui()

Output()